<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-26 13:09:44
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.36 C
-------------------
Today PnL: -93.89 K (-0.68%)
Current PnL: -19.06 L (-12.96%)
CY Booked + Current PnL: -10.33 L (-7.02%)
-------------------
Total profit:  3.40 L
Total loss:  -22.46 L
-------------------
Total Booked + Current PnL: 16.65 L (13.87%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.41%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 78.17 L (57.36%)
Deployed:  1.20 C
Current:  1.36 C
CAGR/XIRR %: 8.01%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,HINDUNILVR,2413.81,2723.00,-4.12,H-LC,8.23,301560.0,31213.0,3408.0,2.31,11.55,1.13,12.81,24.0,9.16,2.27,26.12,XY25,NTT,FMCG
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.36,112928.0,8616.0,8029.0,-1.40,8.26,7.11,15.96,9.0,1.07,0.85,26.55,X5K,ATH,METALS
55,MEDANTA,1087.93,1486.00,12.96,H-MC,14.64,161240.0,35040.0,11142.0,-0.45,27.77,6.91,36.59,108.0,3.14,1.21,43.81,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,7098.95,8285.00,-14.53,H-LC,8.11,170764.0,14587.0,11509.0,-1.13,9.34,6.74,16.71,22.0,1.27,1.29,28.41,X40N,BTT,HEALTHCARE
68,SBIN,760.30,863.00,-13.25,M-LC,4.06,215591.0,14112.0,13108.0,-0.36,7.00,6.08,13.51,60.0,1.08,1.62,20.48,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,KPIGREEN,497.21,731.35,5.95,H-SC,11.09,126050.0,753.0,58248.0,-1.92,0.60,46.21,47.09,141.0,0.01,0.95,57.76,MH,ATH,POWER
61,RAJOOENG,102.50,143.33,-31.84,H-SC,10.02,103250.0,750.0,40082.0,-0.67,0.73,38.82,39.83,114.0,0.02,0.78,2.23,AR,ATH,MISC
56,NATIONALUM,189.63,247.44,-50.29,H-MC,1.97,99645.0,-859.0,31498.0,-1.26,-0.85,31.61,30.49,79.0,-0.03,0.75,31.27,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-39.61,H-LC,3.09,145618.0,-4314.0,18916.0,0.18,-2.88,12.99,9.74,4.0,-0.23,1.10,3.18,X40,NTT,FMCG
53,LTIM,5564.16,7230.2,-4.62,H-LC,3.17,189144.0,-11166.0,71137.0,-0.74,-5.57,37.61,29.94,16.0,-0.16,1.42,30.67,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-10.90,H-LC,3.39,274388.0,8962.0,47826.0,1.70,3.38,17.43,21.40,11.0,0.19,2.06,10.95,XY25,NTT,FMCG
21,CIPLA,1492.70,1795.0,-18.41,H-LC,4.24,217446.0,12946.0,28464.0,-0.87,6.33,13.09,20.25,10.0,0.45,1.64,15.62,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-12.67,H-LC,6.47,170401.0,-30985.0,118599.0,0.52,-15.39,69.60,43.51,27.0,-0.26,1.28,18.62,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BRITANNIA,5502.66,6362.67,14.53,H-LC,8.69,207234.0,9138.0,21822.0,3.71,4.61,10.53,15.63,26.0,0.42,1.56,27.47,XY25,ATH,FMCG
30,GREENPANEL,375.16,537.00,197.16,M-SC,5.97,132687.0,-50391.0,129370.0,2.55,-27.52,97.50,43.14,230.0,-0.39,1.00,24.81,XY24,NTT,MISC
37,HINDUNILVR,2413.81,2723.00,-4.12,H-LC,8.23,301560.0,31213.0,3408.0,2.31,11.55,1.13,12.81,24.0,9.16,2.27,26.12,XY25,NTT,FMCG
69,SFL,1016.35,1287.00,41.82,M-SC,10.33,187218.0,-75000.0,144832.0,2.15,-28.60,77.36,26.63,229.0,-0.52,1.41,16.40,XY24,NTT,MISC
73,SONACOMS,578.05,1006.00,-35.37,M-SC,14.06,80369.0,-20790.0,95679.0,1.80,-20.55,119.05,74.03,202.0,-0.22,0.60,13.33,AR,BTT,AUTO


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANDHANBNK,214.25,400.00,-11.26,H-SC,8.49,192653.0,-55877.0,271352.0,-3.13,-22.48,140.85,86.70,151.0,-0.21,1.45,28.61,XY24,NTT,BANKS
88,VALIANTORG,512.64,838.00,-307.74,H-SC,13.12,88270.0,-45016.0,129607.0,-2.94,-33.77,146.83,63.47,139.0,-0.35,0.66,47.96,XR,NTT,CHEMICALS
87,VAIBHAVGBL,350.21,521.00,36.64,H-SC,11.09,91097.0,-60894.0,135015.0,-2.67,-40.06,148.21,48.77,125.0,-0.45,0.69,6.03,XR,NTT,JEWELLERY
85,UNIONBANK,123.87,163.00,-17.83,M-LC,11.40,147253.0,6413.0,38080.0,-2.65,4.55,25.86,31.59,66.0,0.17,1.11,32.25,XY24,NTT,BANKS
71,SIEMENS,4406.90,7969.85,-4.11,H-LC,33.17,77038.0,-33134.0,122213.0,-2.63,-30.08,158.64,80.85,15.0,-0.27,0.58,13.46,AR,ATH,ELECTRICAL


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,95.94,M-SC,8.62,84443.0,-16334.0,16424.0,-1.76,-16.21,19.45,0.09,245.0,-0.99,0.64,10.94,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,85.53,H-SC,15.48,117396.0,-25083.0,44599.0,-0.62,-17.60,37.99,13.70,163.0,-0.56,0.88,38.56,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-19.81,H-SC,3.04,114390.0,-29413.0,56165.0,-1.73,-20.45,49.10,18.60,149.0,-0.52,0.86,26.23,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,110.38,M-SC,1.68,138012.0,-36547.0,36573.0,-1.19,-20.94,26.50,0.02,221.0,-1.00,1.04,20.05,OX40N,NTT,PAINTS
50,KANSAINER,299.63,340.0,-69.25,H-SC,3.64,211455.0,-58212.0,94542.0,0.17,-21.59,44.71,13.47,138.0,-0.62,1.59,6.95,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-25.80,H-LC,15.92,245755.0,-50179.0,144086.0,0.32,-16.96,58.63,31.73,1.0,-0.35,1.85,4.92,X40,BTT,IT
45,INFY,1461.46,2275.00,-16.73,H-LC,11.91,272939.0,11338.0,134286.0,-0.48,4.33,49.20,55.67,3.0,0.08,2.05,10.70,X40,BTT,IT
47,ITC,411.90,452.00,-39.61,H-LC,3.09,145618.0,-4314.0,18916.0,0.18,-2.88,12.99,9.74,4.0,-0.23,1.10,3.18,X40,NTT,FMCG
89,VBL,492.64,671.64,-9.35,H-LC,8.64,275322.0,10282.0,86011.0,-0.82,3.88,31.24,36.33,5.0,0.12,2.07,17.70,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.01,H-LC,12.83,196911.0,-19608.0,112515.0,-1.01,-9.06,57.14,42.91,7.0,-0.17,1.48,3.12,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,12.96,H-MC,14.64,161240.0,35040.0,11142.0,-0.45,27.77,6.91,36.59,108.0,3.14,1.21,43.81,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-7.52,H-LC,11.15,230966.0,46485.0,23882.0,-0.25,25.20,10.34,38.14,38.0,1.95,1.74,38.47,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.96,H-MC,14.64,161240.0,35040.0,11142.0,-0.45,27.77,6.91,36.59,108.0,3.14,1.21,43.81,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,8.25,H-MC,4.81,186800.0,22130.0,45841.0,-1.91,13.44,24.54,41.28,89.0,0.48,1.41,22.49,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,5.95,H-SC,11.09,126050.0,753.0,58248.0,-1.92,0.60,46.21,47.09,141.0,0.01,0.95,57.76,MH,ATH,POWER
90,VOLTAS,1278.28,1918.49,-3.74,H-MC,6.76,202065.0,10323.0,85716.0,-0.76,5.38,42.42,50.08,99.0,0.12,1.52,12.55,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,14.06,178444.0,27531.0,29782.0,0.68,18.24,16.69,37.97,93.0,0.92,1.34,48.16,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,14.06,178444.0,27531.0,29782.0,0.68,18.24,16.69,37.97,93.0,0.92,1.34,48.16,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-49.79,H-SC,12.37,139973.0,16637.0,117129.0,0.13,13.49,83.68,108.45,119.0,0.14,1.05,38.86,AR,ATH,MISC
92,WIPRO,243.46,420.00,-10.62,M-LC,7.74,156885.0,5940.0,103513.0,-0.32,3.94,65.98,72.51,53.0,0.06,1.18,10.19,XR,NTT,IT
91,WHIRLPOOL,1219.98,2270.00,-43.06,M-SC,5.57,96225.0,4726.0,74026.0,-1.99,5.17,76.93,86.07,223.0,0.06,0.72,39.20,XR,NTT,DURABLES
61,RAJOOENG,102.50,143.33,-31.84,H-SC,10.02,103250.0,750.0,40082.0,-0.67,0.73,38.82,39.83,114.0,0.02,0.78,2.23,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,27.09,82253.0,-31296.0,71346.0,-0.35,-27.56,86.74,35.27,268.0,-0.44,0.62,101.32,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-3.16,H-SC,16.16,90059.0,-10904.0,105432.0,-1.32,-10.80,117.07,93.63,148.0,-0.10,0.68,34.14,SR,ATH,CHEMICALS
54,MASFIN,326.60,398.61,-18.25,H-SC,12.56,93045.0,-4935.0,26536.0,-0.77,-5.04,28.52,22.05,152.0,-0.19,0.70,35.25,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.36,112928.0,8616.0,8029.0,-1.40,8.26,7.11,15.96,9.0,1.07,0.85,26.55,X5K,ATH,METALS
84,UJJIVANSFB,52.77,60.00,85.53,H-SC,15.48,117396.0,-25083.0,44599.0,-0.62,-17.60,37.99,13.70,163.0,-0.56,0.88,38.56,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,27.09,82253.0,-31296.0,71346.0,-0.35,-27.56,86.74,35.27,268.0,-0.44,0.62,101.32,XR,NTT,HOTELS
28,DMART,3764.92,5201.00,-7.52,H-LC,11.15,230966.0,46485.0,23882.0,-0.25,25.20,10.34,38.14,38.0,1.95,1.74,38.47,X40N,NTT,FMCG
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,14.06,178444.0,27531.0,29782.0,0.68,18.24,16.69,37.97,93.0,0.92,1.34,48.16,AR,ATH,AUTO
37,HINDUNILVR,2413.81,2723.00,-4.12,H-LC,8.23,301560.0,31213.0,3408.0,2.31,11.55,1.13,12.81,24.0,9.16,2.27,26.12,XY25,NTT,FMCG
69,SFL,1016.35,1287.00,41.82,M-SC,10.33,187218.0,-75000.0,144832.0,2.15,-28.60,77.36,26.63,229.0,-0.52,1.41,16.40,XY24,NTT,MISC


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.01
1,25,41.66
2,50,71.29


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.20
LC    36.54
MC    26.26
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.36
X40      14.44
X40N     13.95
XY25     11.95
XR        8.97
AR        8.45
OX40N     7.38
X5K       2.27
MH        1.70
X200      1.42
SR        1.11
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.31
H-MC    23.38
H-SC    22.45
M-SC    13.20
M-LC     7.19
M-MC     2.55
L-SC     1.55
L-LC     1.04
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.74,0.58,29.87
IT,12.08,-16.16,74.17
FINANCE,8.00,-22.68,68.04
BANKS,7.44,-15.35,64.32
MISC,5.96,-16.34,83.03
PAINTS,5.52,-18.94,42.37
ELECTRICAL,4.72,-11.96,60.45
AUTO,4.57,-11.14,59.49
AC,3.66,0.45,33.27


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2867128.0,23
XR,1101312.0,13
AR,1084399.0,10
X40,818719.0,11
X40N,514722.0,11
OX40N,478487.0,10
XY25,457193.0,8
SR,196279.0,2
X5K,137637.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2790961.0,27
M-SC,1492275.0,17
H-MC,1270466.0,18
H-LC,1140290.0,19
M-LC,423342.0,5
M-MC,338421.0,2
L-SC,256341.0,3
L-MC,59218.0,1
L-LC,45445.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      944904.0      6
M-SC       XY24      870205.0      7
H-SC       AR        661841.0      5
           XR        530431.0      6
H-LC       X40       509823.0      6
H-MC       XY24      487380.0      5
M-MC       XY24      338421.0      2
H-LC       AR        234728.0      2
H-MC       X40       223606.0      4
H-SC       OX40N     206324.0      4
H-MC       XY25      201489.0      2
H-SC       SR        196279.0      2
H-LC       X40N      194412.0      5
H-SC       X40N      192934.0      3
M-LC       XY24      177113.0      2
L-SC       XR        166385.0      2
M-SC       AR        158048.0      2
           OX40N     131014.0      4
M-LC       X5K       129608.0      1
H-MC       X40N      127376.0      3
M-SC       XY25      124095.0      1
           XR        123623.0      2
H-MC       XR        118142.0      1
M-LC       XR        103513.0      1
L-SC       OX40N      89956.0      1
M-SC       X40        85290.0      1
H-LC       XY25       73056.0      3
           X200       71137.0      1
L-MC       XR         59218.0      1
H-SC       MH         58248.0      1
H-MC       OX40N      51193.0      1
H-LC       XY24       49105.0      1
L-LC       XY25       45445.0      1
H-MC       MH         31498.0      1
           AR         29782.0      1
M-LC       XY25       13108.0      1
H-LC       X5K         8029.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
